# Contador de Cruces e Intersecciones

Mediante esta actividad se analiza la posibilidad de detectar y contabilizar cruces en distintos tipos de imagen.
Para ello se aprovecha la capacidad de suavizado y redondeo de ángulos mediante operadores morfológicos de Apertura y Cierre, engrosando las intersecciones y facilitando su posterior aislamiento eliminando las conexiones y puentes.

La investigación se presenta a través de una interfaz gráfica Tkinter puesto que algunas operaciones precisan supervisión y análisis en tiempo real. Se exponen dos MODOS, uno MANUAL y otro AUTOMÁTICO.



In [1]:
from tkinter import *
from PIL import Image, ImageTk
import numpy as np
import cv2
import matplotlib.pyplot as plt 
import skimage
from skimage import io
from scipy import stats as st
from skimage import img_as_ubyte
from skimage.color import rgb2gray
from skimage.morphology import skeletonize
from skimage.util import img_as_float
global pruebap
pruebap=0
global_compare=0 #usaremos esta variable para la funcion que compara la imagen modificada con la original,
                #a modo de flag para volver a tener la imagne modificada
import networkx as nx                       
import networkx as nx   # Libreria para grafos
global angulo_umbral
angulo_umbral=45        # ángulo máximo en grados entre vectores directores de rectas considerados colineales
global energia
energia=190             # Polilinea de un segmento sobre un mapa umbralizado ¿cuánto coincide la intersección? (0-255)
global origen, fin
origen, fin= 2, 3       # Nodos Origen y Fin para calcular rutas predefinidas. fin se ajustará en realidad a n-2 en la función
                        # Se pueden modificar siempre con origen >=0, fin<= nº de nodos -1, y origen < fin

# MODO MANUAL

El modo manual exige una supervisión de usuario elevada. Las intervenciones en la imagen son fácilmente analizables por el usuario, asemejándose su funcionamiento a un modelo de caja blanca. Permite reajuste y puede emplearse en múltiples tipologías de imagen. Para procesamientos más rápidos se recomienda el modo automático.

El funcionamiento del modo manual se resume en:

1.- UMBRALIZACIÓN: Se umbraliza la imagen manualmente con el deslizador en pantalla. Este paso es común para modo manual y automático. 

2.- APERTURA O CIERRE: Se recomienda ajustar un tamaño de kernel medio o grande utilizando el deslizador en pantalla. Nótese que el kernel propuesto es de tamaño circular, ya que este es el que mejor se adapta a diversos típos de ángulos y encuentros. Una vez ajustado ha de realizarse una dilatación + erosión o viceversa (en función de la disposición del foreground o background), lo que correspondería a una apertura o cierre. 

3.- EROSIONES O DILATACIONES SUCESIVAS: Una vez realizada la apertura o cierre, se recomienda ajustar el tamaño de kernel pequeño o medio, y posteriormente realizar erosiones o dilataciones sucesivas (en función del foreground o background) hasta eliminar los puentes y conexiones. 

4.- CONTORNOS CANNY: Se utilizan contornos canny para contabilizar las regiones extraídas en nuestro programa. Dicho contador aparece en la parte inferior izquierda de la interfaz. El objetivo es extraer la cantidad de intersecciones de la imagen, pero en el modo manual puede emplearse para otros motivos (unión de objetos, regeneración de intersecciones ficticias, etc).



# Funciones para ejecutar operadores morfológicos en modo manual

Con estas funciones daremos la opción al usuario de ejecutar manualmente los operadores morfológicos que considere oportunos, seleccionado en cada momento a través de un slider el tamaño que considere para el elemento estructural.

In [2]:
def el_estructural (value=None):
    n = sl2.get()
    p = (n*2)+1
    window.se = np.zeros((p,p), np.uint8)
    cv2.circle(window.se,(n,n), n, 1, -1)
    lab_se.config(text = "Elemento Estructural Circular " + str(p) + "x" + str(p))

def dil_execute():
    window.modified = cv2.dilate(window.modified, window.se)
   
    window.image2 = Image.fromarray(window.modified)
    window.show2  = ImageTk.PhotoImage(window.image2)
    label2.config(image=window.show2)
    recount()


def ero_execute():
    window.modified = cv2.erode(window.modified, window.se)
  
    window.image2 = Image.fromarray(window.modified)
    window.show2  = ImageTk.PhotoImage(window.image2)
    label2.config(image=window.show2)
    recount()

# MODO AUTOMÁTICO

El modo automático reduce la supervisión de usuario y propone otras utilidades para el algoritmo. Aunque algunas intervenciones no son regulables se apuesta por la rapidez y la eficiencia en imágenes de determinado tipo de contenido. Especialmente indicado para mapas y planos, imagenes donde la extracción del background y el foreground es más directa. El usuario no tiene que intervenir en las operaciones morfológicas ya que el programa se encarga de interpretar y realizar las mismas.

El funcionamiento del modo manual se resume en:

1.- UMBRALIZACIÓN: Se umbraliza la imagen manualmente con el deslizador en pantalla. Este paso es común para modo manual y automático. 

2.- SKELETONIZACIÓN: La skeletonización es un método de vectorización implementado en la librería scikit que extrae ejes de grosor 1 píxel de los elementos foreground. Es el proceso principal del modo automático, ya que a través de la skeletonización el tamaño de los kernel y las iteraciones de dilatación y erosión pueden reducirse a un solo caso, y por tanto, automatizarse. 

3.- CRUCES: La extracción de cruces es el método automático análogo a los pasos 2. y 3. del modo manual; extrayendo nodos para ejes de grosor 1 píxel. 

4.- PUNTOS: Extrae los centroides de los cruces obtenidos previamente, ya que dichos cruces pueden tener geometrías que, aunque pequeñas, resulten irregulares. Esta regularización abstrae los cruces a un único píxel (aunque se representa mayor para facilitar su visibilización).

5.- GRAFO + CALLEJERO: Este paso propone una implementación para el algoritmo, extrayendo un grafo que conecta los puntos y limpia la duplicidad de bordes en función a sus distancias. El botón callejero evalua y arroja el camino con el menor número de cruces a recorrer a través de dicho grafo mediante un punto de salida y uno de llegada.

6.- CONTORNOS CANNY: Igualmente al modo manual, el método basado en contornos canny se emplea para contabilizar los cruces extraídos en la imagen.


# Funciones morfológicas para el modo automático:

Con estas funciones, cuando seleccionemos el módo automático realizaremos una búsqueda de cruces en el mapa seleccionado. Adicionalmente se añade funcionalidad extra que permite trazar grafos entre los cruces y encontar una ruta posible, la cual hemos predefinido entre los nodos 2 y n-2.

Se visualizará también por pantalla el número de cruces que encontramos al aplicar las diferentes funciones.



In [3]:
def funcion_skeleton(thresh1):
    '''
    el objetivo principal de la funcion skeleton es conseguir un mapa en el que todas las vias tengan ancho
    1px para hacer más sencillo los procesos morfológicos posteriores. Al tener todas las calles de 1px no
    nos tenemos que preocupar demasiado en si un elemento estructurante servirá para una calle pero no para otra
    
    '''
    
    #para hacer el skeleton metemos un borde negro de tres pixeles a la imagen binaria
    #hacemos esto porque hemos visto que en los bordes de la imgen, el skeleton
    #curva las rectas y puede dar lugar a malas interpretaciones por el algoritmo
    print(thresh1.shape)
    bordepx=3
    top, bottom, left, right = [bordepx]*4
    thresh1 = cv2.copyMakeBorder(thresh1, top, bottom, left, right, cv2.BORDER_CONSTANT, value=0)
    
    #erosionamos para quitar pequeños caminos que no nos interesan:
    kernel = np.ones((2,2),np.uint8)
    thresh1=cv2.erode(thresh1,kernel,iterations =1)
    
    #aplicamos el metodo skeletonize
    skeleton = skeletonize(thresh1,method='lee')
    skeleton2=skeleton.copy()
    print('skeleton shape 0:',skeleton.shape)
    
    #quitamos los bordes que habíamos agregado
    skeleton=skeleton[bordepx:(skeleton.shape[0]-bordepx),bordepx:(skeleton.shape[1]-bordepx)]
    print(thresh1.shape)
    return skeleton
    
def funcion_cruces(imagen):
    
    '''
    esta es la funcion principal donde , a partir de la imagen de skeleton , encontraremos los cruces que
    aparecen en el mapa.
    
    Para ello, primero haremos una dilatación del skeleton para tener las carreteras un poco más anchas pero del
    mismo grosor, a esa dilatación le concatenamos una operación de cierre que nos ensanchará un poco más las zonas
    de los cruces, dejando con una forma prácticamente inalterada las calles:
    '''
    bordepx=3
    top, bottom, left, right = [bordepx]*4
    imagen = cv2.copyMakeBorder(imagen, top, bottom, left, right, cv2.BORDER_CONSTANT, value=0)
    
    
    kernel = np.ones((3,3),np.uint8)
    imagen=cv2.cvtColor(img_as_ubyte(imagen), cv2.COLOR_GRAY2BGR)
    
    dil = cv2.dilate(imagen,kernel,iterations = 1) # dilatacion
    kernel = np.ones((3,3),np.uint8)
    dil = cv2.morphologyEx(dil, cv2.MORPH_CLOSE, kernel)  #clausura
    
    '''
    Una vez tenemos los cruces al grosor que nos permita diferenciarlos, vamos a aplicar un proceso de erosion complejo.
    Para ello tenemos cuatro elementos estructurantes con forma de cuatro esquinas:
    
    
    '''
    
    kernel1=np.array ( [  [1,1,0,0,0],[1,1,0,0,0],[1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1 ]]  ).astype('uint8')
    kernel2=np.array ( [  [0,0,0,1,1],[0,0,0,1,1],[1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1] ]  ).astype('uint8')
    kernel3=np.array ( [  [1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1],[1,1,0,0,0],[1,1,0,0,0]]  ).astype('uint8')
    kernel4=np.array ( [  [1,1,1,1,1],[1,1,1,1,1],[0,0,0,1,1],[0,0,0,1,1],[0,0,0,1,1] ]  ).astype('uint8')
    
    '''
    Aplicamos una erosion sobre la imagen con cada elemento estructurante y luego aplicamos el operador OR dos a dos:
    '''
    
    erosion1=cv2.erode(dil,kernel1,iterations =1)
    erosion2=cv2.erode(dil,kernel2,iterations =1)
    erosion3=cv2.erode(dil,kernel3,iterations =1)
    erosion4=cv2.erode(dil,kernel4,iterations =1)
    erosion = cv2.bitwise_or(erosion1,erosion2)
    erosion2=cv2.bitwise_or(erosion3,erosion4)
    erosion=cv2.bitwise_or(erosion,erosion2)

     

    
    #quitamos algún pixel suelto que nos haya podido quedar:
    kernel=np.array([[1,1]]).astype('uint8')
    erosion=cv2.erode(erosion,kernel,iterations =1)
    
    #dilatamos el resultado para juntar los puntos obtenidos en las erosiones que corresponden a la misma esquina
    kernel = np.ones((3,3),np.uint8)
    dil2= cv2.dilate(erosion,kernel,iterations = 1) 
    
    
    dil2=dil2[bordepx:(dil2.shape[0]-bordepx),bordepx:(dil2.shape[1]-bordepx)]
    return dil2


 
    
    
def recount ():
    global global_compare
    global puntos1
    puntos1=[]
    window.photo2 = np.array(window.image2 )
    window.photo2 = window.photo2.astype(np.uint8)
    
    
    borde = cv2.Canny(window.photo2, 120, 210)
    contornos, jerarquia = cv2.findContours(borde, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    puntos1=centroides(contornos)     # Metido NUEVO
    #lab_cont.config(text = "Número de contornos = " + str(len(contornos)))
    lab_cont.config(text = "Número de contornos = " + str(len(puntos1)))  # Metido NUEVO
    return puntos1    # Metido NUEVO


def umbral (value=None):
    #Con la funcion umbral, además de seleccionar y aplicar el umbral a la imagen original, reiniciaremos todas
    #las operaciones que hayamos ejecutado preivamente
    
    global global_compare
    global_compare=0
    window.modified = original
    umb_value = sl1.get()

    window.modified = np.array(window.modified)
    window.modified[window.modified < umb_value] = 0
    window.modified[window.modified > 0] = 255
    
    window.image2 = Image.fromarray(window.modified)
    window.show2  = ImageTk.PhotoImage(window.image2)
    label2.config(image=window.show2)
    recount()
    


def exec_busqueda():
    #llamamos a la funcion que busca los cruces desde la imagen skeleton y sacamos el resultado por pantalla
    window.modified = funcion_cruces(window.modified )
   
    window.image2 = Image.fromarray(window.modified)
    window.show2  = ImageTk.PhotoImage(window.image2)
    label2.config(image=window.show2)
    recount()


    


def exec_skeleton():
    #llamamos a la funcion que obtiene el esqueleto y la sacamos por pantalla.
    global img_negra
    window.modified = funcion_skeleton(window.modified)
  
    window.image2 = Image.fromarray(window.modified)
    window.show2  = ImageTk.PhotoImage(window.image2)
    label2.config(image=window.show2)
    img_negra=window.modified  # Metido NUEVO
    recount()


In [4]:
def cambia_modo():
    #funcion para cambiar el modo de ejecución (auto/man), mostrando los botoenes que se van a utilizar en cada caso
    global pruebap
    if pruebap==1:
        pruebap=0
        btnskel.grid_forget()
        btncruz.grid_forget()
        btnpuntos.grid_forget()
        btngrafo.grid_forget()
        btncallejero.grid_forget()
        lab_se.grid(column=2, row=1)
        btnd.grid(column=2, row=3)
        btne.grid(column=2, row=4)  
        lab_modo.config(text='Modo MANUAL')
        sl2.grid(column=2, row=2) 

  
    else:
        pruebap=1
        btnskel.grid(column=2, row=2)  
        btncruz.grid(column=2, row=3)
        btnpuntos.grid(column=2, row=4)  
        btngrafo.grid(column=2, row=5) 
        btncallejero.grid(column=2, row=6)
        lab_se.grid_forget()
        btnd.grid_forget()
        btne.grid_forget()
        lab_modo.config(text='Modo AUTOMÁTICO')
        sl2.grid_forget()


# Funciones adicionales para el modo automático

Con estas funciones conseguimos, en el modo automático y una vez obtenidos los cruces, encontrar los caminos posibles entre los nodos, así como llegar a trazar una ruta posible en el mapa (en nuestro caso predefinimos una ruta entre los nodos 2 y n-2)

In [5]:
def centroides(contornos):     # Realiza un cálculo del centroide del contorno devuelto por cv2.findContours
    puntos1=[]
    for contorno in contornos:
        momentos=cv2.moments(contorno)
        if momentos['m00']!=0:
            x=int(momentos['m10']/momentos['m00'])
            y=int(momentos['m01']/momentos['m00'])
        else:                  # Cuando el momento m00 sea cero, devuelve coordenadas de un punto cuaquiera del contorno
            for puntos in contorno:
                for punto in puntos:
                    x, y = punto
        puntos1.append([x,y])
    return puntos1

def dibuja_puntos(puntos1):    # Dibuja los puntos de cruce precisos sobre la imagen original
    global img_final
    lista=puntos1
    for ele in lista:
        img_final[ele[1]-3:ele[1]+3,ele[0]-3:ele[0]+3,:]=(255,0,0)
    io.imsave('solucion.png', img_final)
    return lista

def limpia(rectas2,parcial,lista):  # Elimina las rectas que confluyen en un nodo, con ángulo similar. Conserva la más corta
    dmin,pos=10**8,0
    d=np.zeros(len(parcial))
    for i,filas in enumerate(parcial):
        x1,y1,x2,y2=lista[filas[0]][0],lista[filas[0]][1], lista[filas[1]][0],lista[filas[1]][1]
        d[i]=((x1-x2)**2+(y1-y2)**2)**0.5
        if d[i] <dmin: 
            dmin,pos=d[i],i
    for ii,pareja in enumerate(parcial):
        if ii!=pos: 
            posicion=0
            while posicion<len(rectas2):
                if rectas2[posicion]==[pareja[0],pareja[1]] or rectas2[posicion]==[pareja[1],pareja[0]] :
                    rectas2.pop(posicion)
                else:
                    posicion=posicion+1                    
    return rectas2 

def crea_rectas(lista,sk1):   # Crea las rectas (vértices) que unen todos los puntos de cruce (nodos)
    rectas=[]                 # siempre que existan en origen
    kernel = np.ones((3,3),np.uint8)
    sk1 = cv2.dilate(sk1,kernel,iterations = 3) # dilatacion
    for i in range (len(lista)):#len(lista)
        x1,y1=lista[i][0],lista[i][1]
        for j in range (i+1,len(lista)):#len(lista)
            x2,y2=lista[j][0],lista[j][1]
            mat=sk1.copy()
            mat[:,:]=0
            pts = np.array((x1, y1), np.int32)
            pts = np.append(pts,(x2, y2))
            pts = pts.reshape((-1, 1, 2)) 
            mat = cv2.polylines(mat, [pts], False, (255), 1)
            mat=np.bitwise_and(sk1,mat)
            suma=mat.sum()
            distancia=((x2-x1)**2+(y2-y1)**2)**0.5
            if suma>distancia*energia: # Parámetro configurable entre 0 y 255. ¿Cuánta energía debe coincidir? ej 190
                rectas.append([i,j])
    return rectas

def angulo(v1, v2):                        #Devuelve el ángulo en grados que forman 2 segmentos
    numerador = v1.dot(v2)
    denominador = np.linalg.norm(v1) * np.linalg.norm(v2)
    coseno=numerador/denominador
    if coseno>1: coseno=1
    if coseno<-1: coseno=-1
    if denominador!=0: arco=np.rad2deg(np.arccos(coseno)) 
    else: arco=90
    return arco

def simplifica(rectas,lista):     #simplifica rectas  NUEVO SABADO 30_1_21
    rectas2=rectas.copy()
    for indice1 in range(0,len(lista)-1):
        for indice2 in range (indice1+1, len(lista)):
            for segmento1 in rectas:
                if indice1 in segmento1 and indice2 in segmento1:
                    parcial=np.array([[indice1,indice2]],dtype=(np.int16))
                    for segmento2 in rectas:
                        if segmento1==segmento2: continue
                        if indice1 in segmento2:
                            if segmento2[0]!=indice1:
                                indice3, indice4=segmento2[1],segmento2[0]
                            else:
                                indice3, indice4=segmento2[0],segmento2[1]
                                if indice2==indice4: continue
                            u1=([lista[indice1][0],lista[indice1][1]])
                            u2=([lista[indice2][0],lista[indice2][1]])
                            v1=np.array([u1[0]-u2[0], u1[1]-u2[1]])
                            u3=([lista[indice3][0],lista[indice3][1]])
                            u4=([lista[indice4][0],lista[indice4][1]])
                            v2=np.array([u3[0]-u4[0], u3[1]-u4[1]])
                            angulo1=angulo(v1,v2)
                            if abs(angulo1)<= angulo_umbral: #45:   # Parámetro configurable, ANGULO para eliminar rectas
                                parcial=np.append(parcial,np.array([[indice3,indice4]]),axis=0)
                    if len(parcial)>1:
                        rectas2=limpia(rectas2,parcial,lista)
    return rectas2

def dibuja_grafo(rectas2,lista,sk2):   # Dibuja el grafo con los nodos (cruces hayados) y los vértices (calles)
    rectas=rectas2.copy()
    global imagina
    cerrada,grosor,color = False, 2, (0, 0, 255)
    window_name = 'Grafo Callejero'
    for indice in range(0,len(rectas)):
        aa=rectas[indice][0]
        u1=([lista[aa][0],lista[aa][1]])
        aa=rectas[indice][1]
        u2=([lista[aa][0],lista[aa][1]])
        pts = np.array([u1, u2], np.int32) 
        pts = pts.reshape((-1, 1, 2)) 
        imagina = cv2.polylines(sk2, [pts], cerrada, color, grosor)
    
def grafo_dijkstra(rectas,lista):  # Ejecuta el algoritmo dijkstra sobre el grafo obtenido del mapa
    g=nx.Graph()
    fin=  len(lista)-2  # Parámetros variables Origen y llegada del camino
    for ele in rectas:
        g.add_edge(ele[0],ele[1],d=1)
    nx.dijkstra_path_length(g,origen,fin,'d')
    camino=nx.dijkstra_path(g,origen,fin,'d')
    print("Cruces: %3d Nodos, entre Nodo origen nº:, %3d y fin nº: %3d hay camino óptimo con sólo  %3d Cruces " %(len(lista),origen, fin,len(camino)))
    return camino

def dibuja_ruta(camino, lista,sk2):  # Muestra el resultado de la búsqueda en panatalla
    global imagina1
    cerrada, grosor, color = False, 2, (0, 0, 255) 
    for indice in range (len(camino)-1):
        aa=camino[indice]
        u1=([lista[aa][0],lista[aa][1]])
        aa=camino[indice+1]
        u2=([lista[aa][0],lista[aa][1]])
        pts = np.array([u1, u2], np.int32) 
        pts = pts.reshape((-1, 1, 2)) 
        imagina1 = cv2.polylines(sk2, [pts], cerrada, color, grosor)
        if indice==0:  # marca los nodos de inicio y Fin
            imagina1 = cv2.circle(imagina1, (u1[0],u1[1]), 10, (255,0,0), 2)
        elif indice==len(camino)-2:
            imagina1 = cv2.circle(imagina1, (u2[0],u2[1]), 10, (255,0,0), 2)

def callejero_execute1():
    global img_final
    img_final = io.imread(file)
    lista=dibuja_puntos(puntos1)
    window.image2 = Image.fromarray(img_final)
    window.show2  = ImageTk.PhotoImage(window.image2)
    label2.config(image=window.show2)

def callejero_execute2():
    global img_final
    global imagina
    img_final = io.imread(file)
    lista=dibuja_puntos(puntos1)
    rectas=crea_rectas(lista,img_negra.copy())
    rectas2=simplifica(rectas,lista)
    dibuja_grafo(rectas2,lista,img_final.copy())
    window.image2 = Image.fromarray(imagina)
    window.show2  = ImageTk.PhotoImage(window.image2)
    label2.config(image=window.show2)

    
def callejero_execute3():
    global img_final
    global imagina1
    img_final = io.imread(file)
    lista=dibuja_puntos(puntos1)
    rectas=crea_rectas(lista,img_negra.copy())
    rectas2=simplifica(rectas,lista)
    camino=grafo_dijkstra(rectas2,lista)
    dibuja_ruta(camino, lista,img_final.copy())
    window.image2 = Image.fromarray(imagina1)
    window.show2  = ImageTk.PhotoImage(window.image2)
    label2.config(image=window.show2)

In [11]:
window = Tk()
window.title("Contador de Cruces")
file=(input("Introduce la imagen a analizar con extensión:"))  # Nuevo

original = cv2.imread(file,0) 

imagen_original=original.copy()
original_col = Image.open(file)
window.modified = original

window.image1 = Image.fromarray(original)
window.image2 = Image.fromarray(original)
window.show1 = ImageTk.PhotoImage(original_col)
window.show2 = ImageTk.PhotoImage(window.image2)

window.photo2 = np.array(window.image2 )
window.photo2 = window.photo2.astype(np.uint8)

label1 = Label(image=window.show1)
label1.grid(column=0, row=0)

label2 = Label(image=window.show2)
label2.grid(column=2, row=0)

#----------------------------------------------------------------
borde = cv2.Canny(original, 120, 210)

contornos, jerarquia = cv2.findContours(borde, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

lab_umb = Label(text = "Umbralización (reinicio):", height = 1)
lab_umb.grid(column=0, row=1, columnspan=1)

sl1 = Scale(from_=0, to=255, length=200, command=umbral, orient="horizontal", showvalue = 0, width = 20)
sl1.grid(column=0, row=2, columnspan=1,sticky="n")

lab_modo = Label(text = "Selecciona Modo: ", height = 2)
lab_modo.grid(column=0, row=3)

btnMODO = Button(text="MODO", command=cambia_modo, height = 1, width = 10)
btnMODO.grid(column=0, row=4) 

lab_cont = Label(text = "Número de contornos = " + str(len(contornos)), height = 2)
lab_cont.grid(column=0, row=8,  columnspan=1)

window.se = np.zeros((3,3), np.uint8)
cv2.circle(window.se,(1,1), 5, 1, -1)


#botones modo auto:

btnskel = Button(text="Skeleton", command=exec_skeleton, height = 1, width = 10)
btncruz = Button(text="Cruces", command=exec_busqueda, height = 1, width = 10)
btnpuntos = Button(text="Puntos", command=callejero_execute1, height = 1, width = 10)   # Metido NUEVO
btngrafo = Button(text="Grafo", command=callejero_execute2, height = 1, width = 10)    # Metido NUEVO
btncallejero = Button(text="Callejero", command=callejero_execute3, height = 1, width = 10) # Metido NUEVO

#botones modo manual:

btnd = Button(text="Dilatar", command=dil_execute, height = 1, width = 10)
btne = Button(text="Erosionar", command=ero_execute, height = 1, width = 10)
lab_se = Label(text = "Elemento Estructural Circular 3x3", height = 1)
sl2 = Scale(from_=1, to=5, length=200, command=el_estructural, orient="horizontal", showvalue = 0, width = 20)


window.mainloop()   

Introduce la imagen a analizar con extensión:mapa2.jpg
(402, 423)
skeleton shape 0: (408, 429)
(408, 429)
Cruces: 115 Nodos, entre Nodo origen nº:,   2 y fin nº: 113 hay camino óptimo con sólo   12 Cruces 
